<a href="https://colab.research.google.com/github/SEOYUNJE/Lung-Image-Analysis/blob/main/Segmentation/utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader


class MaskGenerator(Dataset):
    def __init__(self, data, preprocess=None):
        self.data = data
        self.preprocess = preprocess
        self.clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        path = self.data.iloc[index]['path']
        img = Image.open(path).convert('L')
        img = img.resize((256,256), Image.Resampling.LANCZOS)
        img = np.array(img)
        img = self.clahe.apply(img)
        img = np.expand_dims(img, axis=-1)
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)  # Convert grayscale to RGB

        img = torch.from_numpy(img).permute(2,0,1).to('torch.float32')

        return img


In [ ]:
dataset = MaskGenerator(df_train)
dataloader = DataLoader(dataset, batch_size=16, shuffle=False, num_workers=2, drop_last=False, )

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device is ',device)

In [ ]:
%%time
seg_model = loadModel(device)

# seg_model.to(device).eval()
# 이미 loadModel에 eval, device 다 있음

all_mask = []
with torch.no_grad(): # 추론 모드
    for batch in tqdm(dataloader):

        batch = batch.to(device) # cpu -> gpu로 이동
        oof = seg_model(batch).cpu().numpy() # torch -> numpy

        all_mask.append(oof)

del seg_model
gc.collect()

all_mask = np.concatenate(all_mask)